In [ ]:
# Import necessary libraries for numerical operations, plotting, and image processing.
import math
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import RegularGridInterpolator
from scipy import interpolate
import cv2
import pandas as pd
from sklearn.metrics import r2_score
from scipy.optimize import curve_fit

# Import and mount Google Drive to access files.
from google.colab import drive
drive.mount('/content/drive')

# Set font parameters for matplotlib plots.
font = {'family':'DejaVu Sans',
        'weight':'normal',
        'size':10,
       }
labsize = 13

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# This cell defines several functions used for calculating lengths, finding points, and fitting curves.

def calculate_path_length(s):
    """
    Calculates the cumulative length along a path defined by a series of points.
    Args:
        s (np.array): An array of points (coordinates) that define the path.
    Returns:
        tuple: A tuple containing a list of cumulative lengths and the total length.
    """
    cumulative_lengths  = []  # List to store cumulative lengths
    total_length = 0  # Initialize total length
    # Iterate through the points to calculate the distance between each point and the next
    for i in range(1, len(s)):
        # Calculate the Euclidean distance between consecutive points and add to the total length
        total_length = total_length + np.linalg.norm(s[i,:] - s[i-1,:])
        cumulative_lengths.append(total_length)  # Append the cumulative length to the list
    return cumulative_lengths, total_length

# Function to get corresponding points on the cell contour.
def get_contour_points(xy, cxy, dcxy):
    """
    Finds the corresponding points on a contour 'xy' for each point on a centerline 'cxy'.
    Args:
        xy (np.array): The contour points.
        cxy (np.array): The centerline points.
        dcxy (np.array): The direction vectors of the centerline.
    Returns:
        tuple: A tuple containing the reordered contour points and the indices of the corresponding points.
    """
    # Calculate the vector from each contour point to the start and end of the centerline
    lb = xy - cxy[0,:]
    lt = xy - cxy[-1,:]
    # Find the indices of the contour points closest to the start and end of the centerline
    ib = np.argmin(np.linalg.norm(lb, axis=1))
    it = np.argmin(np.linalg.norm(lt, axis=1))
    # Reorder the contour points so that the point closest to the start of the centerline is at the beginning
    xy = np.roll(xy, -ib, axis=0)

    # Initialize an array to store the indices of the corresponding points on the contour
    points = np.zeros(dcxy.shape)

    # For each point on the centerline, find the two points on the contour that are "most perpendicular"
    for i in range(1, len(cxy)):
        l = []
        v = xy - cxy[i,:]  # Vector from each contour point to the current centerline point
        for j in range(len(v)):
            # Calculate the absolute value of the dot product to find the perpendicular distance
            l.append(abs(np.dot(v[j,:], dcxy[i-1,:])))
        # Find the indices of the two points with the minimum dot product (most perpendicular)
        points[i-1, 0] = np.argmin(l[0:it-ib])
        points[i-1, 1] = np.argmin(l[it-ib:]) + it-ib
    return xy, points

# Function for linear interpolation between two points.
def linear_interpolate(xy1, xy2, N):
    """
    Performs linear interpolation between two points.
    Args:
        xy1 (list): The coordinates of the first point.
        xy2 (list): The coordinates of the second point.
        N (int): The number of points to generate between xy1 and xy2.
    Returns:
        np.array: An array of N points interpolated between xy1 and xy2.
    """
    x = np.linspace(xy1[0], xy2[0], N)  # Generate N evenly spaced x-coordinates
    f3d = interpolate.interp1d([xy1[0], xy2[0]], [xy1[1], xy2[1]])  # Create a 1D interpolation function
    return np.vstack((x, f3d(x))).T  # Return the interpolated points

# Gaussian function for fitting. 'mu' is defined globally in the main loop.
def gaussian_function(x, amp, sigma, b):
    """
    Gaussian function for curve fitting.
    Args:
        x (float): The independent variable.
        amp (float): The amplitude of the Gaussian.
        sigma (float): The standard deviation of the Gaussian.
        b (float): The baseline offset.
    Returns:
        float: The value of the Gaussian function at x.
    """
    # Note: 'mu' is used here but not defined as a parameter. It is a global variable or passed as an argument.
    return amp * np.exp(-(x - mu)**2 / (2 * sigma**2)) + b

In [ ]:
# This cell sets up the parameters and file paths for the analysis.

# Set the base path for the data files
path = "./drive/MyDrive/Colab Notebooks/2. nucleus(MT band)_detection/"


# --- Parameters ---
t = 231  # Length of the data (number of time points or images)
p = 1/4.8697  # Conversion factor from pixels to micrometers (um/pixel)
row = 512  # Image height in pixels
column = 512  # Image width in pixels

# --- Folder Names ---
# Folder names for input data
folder_nu = "0. images"  # Folder containing data for the nucleus channel
folder_cell_contour = "1. contour"  # Folder containing data for the smoothed cell contour
folder_centerline = "2. centerline"  # Folder containing data for the centerline

# Folder name for output data
folder_ouput = "3. Nucleus Position and Width"  # Folder where the results (nucleus position and width) will be saved

In [ ]:
# Define the coordinate grid for interpolation
iy = np.linspace(0, row - 1, row)
ix = np.linspace(0, column - 1, column)

# Initialize result container
md = []

# --- Parameters ---
threshold = 0.7  # Threshold (0~1) for detecting nucleus position based on brightness
scc = 1.3        # Scaling parameter for estimating nucleus width from Gaussian sigma

# --- Main Processing Loop ---
for i in range(0, int(t)):  # Loop over each time frame
    # --- Step 1: Load image and normalize ---
    impth = path + folder_nu + "/" + "%04d" % i + ".png"
    Im = cv2.imread(impth, 0)  # Read nucleus image (grayscale)
    # Im = Im / np.max(Im) * 255  # Normalize intensity to [0, 255]

    # Create interpolator from image (used later for intensity sampling)
    f_scipy = RegularGridInterpolator((ix, iy), Im)

    # --- Step 2: Load contour and centerline data ---
    cpth = path + folder_cell_contour + "/" + "%04d" % i + ".txt"
    yx = np.loadtxt(cpth)  # Load cell contour (Y-X format)

    rpth = path + folder_centerline + "/" + "%04d" % i + ".txt"
    cyx = np.loadtxt(rpth)  # Load centerline coordinates

    # --- Step 3: Calculate cumulative length along centerline ---
    sx, sl = calculate_path_length(cyx * p)  # sx: position along centerline, sl: total length
    sx = np.array(sx)

    # --- Step 4: Align contour with centerline and compute projections ---
    dcyx = cyx[1:, :] - cyx[:-1, :]  # Direction vectors along centerline
    yx, points = get_contour_points(yx, cyx, dcyx)  # Get matching points on contour

    # --- Step 5: Project image intensity along paths defined by points ---
    mtmpMTs = []
    for ip in range(len(points)):
        tmpyx = f(yx[int(points[ip, 0])], yx[int(points[ip, 1])], 100)
        mtmpMTs.append(np.mean(f_scipy(tmpyx)))  # Mean intensity of each projected slice

    tmpf = mtmpMTs / np.max(mtmpMTs)  # Normalize projection brightness

    # --- Step 6: Fit Gaussian and determine nucleus position & width ---
    mu = np.mean(sx[tmpf >= threshold])  # Estimate center using thresholded region
    popt, pcov = curve_fit(gaussian_function, sx, tmpf)  # Fit Gaussian
    y = gaussian_function(sx, *popt)  # Get fitted curve values

    # Save results: [center position, width, total centerline length]
    mtsbi = path + folder_ouput + "/" + "%04d" % i + ".txt"
    nucleus_center = mu
    nucleus_width = 2 * scc * np.abs(popt[1])  # 2 * scc * sigma
    np.savetxt(mtsbi, [nucleus_center, nucleus_width, sl])

    # --- Step 7: Visualization ---
    fig, (ax2, ax1) = plt.subplots(1, 2, gridspec_kw={'width_ratios': [3, 1]})

    # Plot brightness vs. centerline position
    ax1.plot(tmpf, sx, color='g', linewidth=1)
    ax1.plot(y, sx, '-.', color='k', linewidth=1)

    # Plot estimated width (±scc * sigma range)
    x1 = gaussian_function(mu - scc * np.abs(popt[1]), *popt)
    x2 = gaussian_function(mu + scc * np.abs(popt[1]), *popt)
    ax1.plot([x1, x2], [mu - scc * np.abs(popt[1]), mu + scc * np.abs(popt[1])], color='magenta', linewidth=3)

    # Compute R^2 for fitted curve
    # r2 = r2_score(tmpf[tmpf >= popt[2]], y[tmpf >= popt[2]])
    # ax1.set_title(r'$r^{2}=$' + "%.3f" % r2, fontsize=15)
    ax1.set_xlim([0, 1.1])
    ax1.set_ylim([0, sl])
    ax1.tick_params(labelsize=labsize)

    # Overlay centerline region on original image
    is1 = np.argmin(np.abs(sx - (mu - scc * np.abs(popt[1]))))
    is2 = np.argmin(np.abs(sx - (mu + scc * np.abs(popt[1]))))
    ax2.imshow(Im, cmap="gray")
    ax2.plot(cyx[is1:is2+1, 1], cyx[is1:is2+1, 0], color='magenta', linewidth=1)
    ax2.set_title('Frame=' + "%04d" % i, fontsize=15)
    ax2.axis("off")

    # Save figure
    smf = path + folder_ouput + "/png/" + "%04d" % i + ".png"
    plt.savefig(smf, bbox_inches='tight', dpi=300)
    plt.show()


Output hidden; open in https://colab.research.google.com to view.